In [1]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path="../../.env")
api_key = os.getenv("API_KEY")

In [4]:
embeddings_model = OpenAIEmbeddings(api_key=api_key)
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
from langchain.memory import VectorStoreRetrieverMemory
# 리트리버 설정: k=1로 가장 관련성 높은 1개 문서를 반환하도록 지정
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


# VectorStoreRetrieverMemory 객체 생성
memory = VectorStoreRetrieverMemory(retriever=retriever)
# 임의의 대화 4턴 저장
memory.save_context(
    inputs={"human": "안녕하세요, 오늘 면접에 참석해 주셔서 감사합니다. 자기소개 부탁드립니다."},
    outputs={"ai": "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다."},
)
memory.save_context(
    inputs={"human": "프로젝트에서 어떤 역할을 맡았나요?"},
    outputs={"ai": "제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프런트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다."},
)
memory.save_context(
    inputs={"human": "팀 프로젝트에서 어려움을 겪었던 경험이 있다면..."},
    outputs={"ai": "프로젝트 초기에 의사소통 문제로 몇 가지 어려움이 있었지만, 정기적인 팀 회의와 명확한 역할 분담을 통해 이를 극복하고 성공적으로 프로젝트를 완료했습니다."},
)
memory.save_context(
    inputs={"human": "개발자로서 자신의 강점은 무엇이라고 생각하나요?"},
    outputs={"ai": "제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술을 배우고 적용하는 데 주저함이 없으며, 복잡한 문제에 직면했을 때 논리적으로 접근하여 해결책을 찾는 것을 즐깁니다."},
)

# 쿼리 구문을 통해 가장 관련성 높은 대화 1개 턴 검색
print(memory.load_memory_variables({"human": "면접자 전공은 무엇 인가요?"})["history"])
print("---")
print(memory.load_memory_variables({"human": "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"})["history"])


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
/tmp/ipykernel_23175/310630537.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=retriever)


human: 안녕하세요, 오늘 면접에 참석해 주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.
---
human: 프로젝트에서 어떤 역할을 맡았나요?
ai: 제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프런트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다.


In [1]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever(
    lang="ko",
    doc_content_chars_max=100,
    top_k_results=3
)

In [3]:
result = retriever.invoke("러우 전쟁")
result

[Document(metadata={'title': '러시아-우크라이나 전쟁', 'summary': '러시아-우크라이나 전쟁(러시아어: российско-украинская война, 우크라이나어: російсько-українська війна) 또는 러우 전쟁은 2014년 2월부터 시작된 분쟁으로, 우크라이나와 침략국 러시아 및 우크라이나 내 친러 분리주의 세력 간의 분쟁을 뜻한다. 여러 국가가 우크라이나에게 다양한 수준의 지원을 제공하였으며, 벨라루스는 2022년 러시아의 우크라이나 전면 침공 당시 벨라루스 영토 내 러시아군의 통행을 제공하였다. 2014년 우크라이나 혁명 직후 러시아가 적대 행위를 시작하였으며 이는 대부분 크림반도와 돈바스의 정치적 지위 문제에 집중되었는데 돈바스의 경우에는 국제적으로 우크라이나의 영토로 인정받고 있다. 우크라이나 영토 침공 행위는 러시아의 크림반도 합병으로 절정에 달했고, 그로부터 얼마 지나지 않아 러시아의 지원을 받는 분리주의 세력과 우크라이나 사이 돈바스 전쟁이 발발하였다. 그 외에도 분쟁 시작 후 8년 동안 해상 충돌, 사이버전, 다양한 정치적 긴장 고조 등 여러 사건이 발생하였다. 2021년 내내 우크라이나 영토 주변 러시아군의 증강으로 양국 간 긴장이 고조되었으며, 2022년 2월 24일 러시아의 우크라이나 침공으로 분쟁이 전면전으로 격화되었다.\n2014년 2월 2일, 당시 우크라이나의 대통령이었던 빅토르 야누코비치가 유럽 연합-우크라이나 공동체 협정 협상을 거부하고 유라시아 경제 연합과 더욱 긴밀히 연계하기로 결정하자 발생한 유로마이단 시위와 이후 일어난 혁명으로 쫓겨났다. 야누코비치가 러시아로 망명한 직후 동부와 남부 우크라이나에서는 친러 봉기가 발발하였다. 같은 시기 아무런 표식이 없는 러시아군이 우크라이나의 크림반도로 이동하여 주요 전략지대와 인프라를 장악하고, 2014년 2월 27일에는 크림의 의회 건물도 점령하였다. 3월 16일에는 러시아로 합병하자는 크림 주민투표를 열고 주민투표가 열린지 이틀 